In [43]:
import ogr,json

In [1]:
import pyspark

In [3]:
sc = pyspark.SparkContext()

In [4]:
rdd = sc.textFile("./data/dltb.json")

### 自定义过滤方法，如果最后一个字符是逗号，则去掉，然后解析json，取出空间部分，构建成几何要素，然后验证是否可用

In [44]:
def MyFilter(line):
    try:
        if line[-1:] == ",":
            line = line[:-1]
        dt = json.loads(line)
        geojson = json.dumps(dt["geometry"])
        return ogr.CreateGeometryFromJson(geojson).IsValid()
    except:
        return False

### 查看原始rdd的前面六行

In [7]:
rdd.take(6)

['{',
 '"type": "FeatureCollection",',
 '"name": "dltb",',
 '"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:EPSG::3857" } },',
 '"features": [',
 '{ "type": "Feature", "id": 0, "properties": { "TBBH": "379", "DLMC": "旱地", "TBMJ": 1489.7, "Shape_Leng": 1375.7685861499999, "Shape_Area": 47227.815217399999 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 11135971.080299999565, 2710491.250100001692772 ], [ 11135968.1246, 2710484.883000001311302 ], [ 11135961.2853, 2710475.23299999908 ], [ 11135951.616799999028, 2710465.415800001472235 ], [ 11135948.7961, 2710463.401999998837709 ], [ 11135948.193799998611, 2710461.321699999273 ], [ 11135956.735399998724, 2710450.2049 ], [ 11135957.76, 2710443.669399999082 ], [ 11135956.4021, 2710439.585 ], [ 11135952.13199999928, 2710436.7181 ], [ 11135946.85339999944, 2710436.924199998378754 ], [ 11135943.8612, 2710438.9107 ], [ 11135932.2478, 2710452.706300001591444 ], [ 11135928.322399999946, 2710457.997000001370907 ], [ 111359

### 原始RDD，一共有199行，那么看看有多少个要素

In [8]:
rdd.count()

199

### 过滤掉之后，就剩下了192个要素了

In [45]:
rdd2 = rdd.filter(lambda line : MyFilter(line))

In [46]:
rdd2.count()

192

In [48]:
rdd2.take(2)

['{ "type": "Feature", "id": 0, "properties": { "TBBH": "379", "DLMC": "旱地", "TBMJ": 1489.7, "Shape_Leng": 1375.7685861499999, "Shape_Area": 47227.815217399999 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 11135971.080299999565, 2710491.250100001692772 ], [ 11135968.1246, 2710484.883000001311302 ], [ 11135961.2853, 2710475.23299999908 ], [ 11135951.616799999028, 2710465.415800001472235 ], [ 11135948.7961, 2710463.401999998837709 ], [ 11135948.193799998611, 2710461.321699999273 ], [ 11135956.735399998724, 2710450.2049 ], [ 11135957.76, 2710443.669399999082 ], [ 11135956.4021, 2710439.585 ], [ 11135952.13199999928, 2710436.7181 ], [ 11135946.85339999944, 2710436.924199998378754 ], [ 11135943.8612, 2710438.9107 ], [ 11135932.2478, 2710452.706300001591444 ], [ 11135928.322399999946, 2710457.997000001370907 ], [ 11135928.1552, 2710461.074 ], [ 11135929.367899999022, 2710463.464899998158216 ], [ 11135930.958799999207, 2710466.78099999949 ], [ 11135930.718299999833, 2710469.1651 ],

### 按地类名称进行统计一下

In [53]:
def myMap(line):
    if line[-1:] == ",":
            line = line[:-1]
    dt = json.loads(line)
    geojson = json.dumps(dt["geometry"])
    return (dt["properties"]["DLMC"],
                ogr.CreateGeometryFromJson(geojson).GetArea())

In [54]:
rdd2.map(lambda js:myMap(js)).reduceByKey(lambda x,y : x+y).collect()

[('旱地', 1062114.5405702349),
 ('园地', 974189.4862148697),
 ('农村居民点用地', 39557.91353731044),
 ('其他独立建设用地', 257737.88656521734),
 ('林地', 4250805.430876931)]